# Regressão linear em Python 🐍

## Obtendo os dados

Para exemplificar o ajuste de regressões lineares será usado o conjunto de dados `diamonds` disponível na biblioteca *Seaborn*. O conjunto de dados mostra dados de `53.940` diamantes com 10 variáveis:

* **price** - preço em dólares americanos (`$326` a `$18.823`).
* **carat** - peso do diamante em quilates (`0,2` a `5.01`).
* **cut** - qualidade do corte (`Fair`, `Good`, `Very Good`, `Premium`, `Ideal`).
* **color** - cor do diamante de `D` (melhor) para `J` (pior).
* **clarity** - uma medida da clareza do diamante (`I1` (pior), `SI2`, `SI1`, `VS2`, `VS1`, `VVS2`, `VVS1`, `IF` (melhor)).
* **x** - comprimento em milímetros (`0` a `10.74`).
* **y** - largura em milímetros  mm (`0` a `58.9`).
* **z** - profundidade em milímetros (`0` a `31.8`).
* **depth** - percentagem total da profundidade = z / mean(x, y) = 2 * z / (x + y) (`43` a `79`).
* **table** - largura do topo do diamante relativo ao ponto mais largo (`43` a `95`).



In [ ]:
import seaborn as sns

dds = sns.load_dataset('diamonds')
dds['cut']     = dds['cut'].astype('category')
dds['color']   = dds['color'].astype('category')
dds['clarity'] = dds['clarity'].astype('category')
dds.head()

Como o preço dos diamantes possui uma escala muito grande será utilizado o logaritmo deste valor. Também como a base de dados é muito grande será selecionado apenas 1000 observações ao acaso.

In [ ]:
import numpy as np

dds['logprice'] = np.log(dds['price'])
dds = dds.sample(n= 1000, replace= False)

### Regressão linear com uma única variável explicativa

In [ ]:
import statsmodels.api as sm
Y = dds['logprice']
X = dds['carat']
X.head()

### Adicionando uma coluna para a constante

Para ajustar o modelo com **intercepto** é necessário adicionar uma coluna constante com o método `add_constant()` da biblioteca *Statsmodels*:

In [ ]:
X = sm.add_constant(X)
X.head()

### Ajustando o modelo

Uma vez construído o vetor das respostas (`Y`) e a matriz de covariáveis (`X`) é possível ajustar o modelo por *mínimos quadrados ordinários*:

In [ ]:
model = sm.OLS(Y, X, missing='drop')
model_result = model.fit()
model_result.summary()

### Diagnósticos da regressão

Da mesma forma que o *R* a biblioteca *Statsmodels* expõe os resíduos do modelo. Uma suposição fundamental do modelo de regressão é que os resíduos (ou “erros”) são aleatórios: os erros seguem uma distribuição Normal com média ZERO.

#### Histograma dos resíduos

Desenhar um histograma para os resíduos com a biblioteca *Seaborn* é trivial: simplesmente forneça os resíduos para o método `histplot()`:

In [ ]:
import seaborn as sns
sns.histplot(model_result.resid);

Uma forma mais útil para verificar a normalidade é comparar o núcleo (*kernel*) da densidade com a curva correspondente à distribuição Normal. Para fazer isso, gena-se uma curva Normal com a mesma média e desvio-padrão dos resíduos.

No *Python* é fácil obter os parâmetros necessários: o método `fit()` retorna a média e o desvio-padrão da distribuição Normal que melhor se ajusta.

In [ ]:
from scipy import stats
mu, std = stats.norm.fit(model_result.resid)
mu, std

Agora é possível desenhar os resíduos com a curva Normal sobreposta:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots()
# plot the residuals
sns.histplot(x=model_result.resid, ax=ax, stat="density", linewidth=0, kde=True)
ax.set(title="Distribution of residuals", xlabel="residual")

# plot corresponding normal curve
xmin, xmax = plt.xlim() # the maximum x values from the histogram above
x = np.linspace(xmin, xmax, 100) # generate some x values
p = stats.norm.pdf(x, mu, std) # calculate the y values for the normal curve
sns.lineplot(x=x, y=p, color="orange", ax=ax)
plt.show()

### *Boxplot* dos resíduos

Um *boxplot* permite verificar a simetria da distribuição.

In [ ]:
sns.boxplot(x=model_result.resid, showmeans=True);

### *Q-Q plot*

Um *Q-Q plot* é um gráfico especializado para mostrar a aderência do modelo com a alguma distribuição. Por padrão a comparação é feita com a distribuição Normal. A biblioteca *Seaborn* define o método `qqplot()` para tal fim.

In [ ]:
sm.qqplot(model_result.resid, line='s');

### Gráfico de ajuste

Um **gráfico de ajuste** mostra os valores preditos pelo modelo e os valores observados para a variável resposta.

In [ ]:
sm.graphics.plot_fit(model_result,1, vlines=False);

Em particular, o modelo está muito mal ajustado. O esperado eram os pontos azuis "em torno" dos pontos vermelhos. Observa-se que os pontos se espalham na base do gráfico.

### Uma segunda opção de gráfico de ajuste.

In [ ]:
model_result.fittedvalues

In [ ]:
Y_max = Y.max()
Y_min = Y.min()

ax = sns.scatterplot(x=model_result.fittedvalues, y=Y)
ax.set(ylim=(Y_min, Y_max))
ax.set(xlim=(Y_min, Y_max))
ax.set_xlabel("Valores preditos para o log-preço por quilate.")
ax.set_ylabel("Valores observados para o log-preço por quilate.")

X_ref = Y_ref = np.linspace(Y_min, Y_max, 100)
plt.plot(X_ref, Y_ref, color='red', linewidth=1)
plt.show()

## Regressões lineares múltipla

Para ajustar o modelo é possível definir dois *data frames* distintos:

1. `Y` para armazenar a variável resposta (uma única coluna "price").
2. `X` para armazenar as variáveis explicativas.

Uma constante é adicionada com o método `add_constant()` da biblioteca *Statsmodels* como na regressão linear simples.

In [ ]:
Y = dds['logprice']
X = dds[['carat', 'x', 'y', 'z', 'depth', 'table']]
X = sm.add_constant(X)

### Modelo inicial

Um primeiro modelo com todas as variáveis numéricas pode ser ajustado para uma análise inicial.

In [ ]:
model = sm.OLS(Y, X)
model_res = model.fit()
model_res.summary()

### Variáveis explicativas categoricas

No *Python* é possível usar tanto a codificação manual (criar uma matriz de variáveis *dummy*) ou a codificação automática (deixar o algoritmo se adaptar aos dados). Em um primeiro passo, observe a codificação manual.

### Criando uma matriz de variáveis *dummy*s

A biblioteca *pandas* possui o método `get_dummies()` para codificar variáveis categorizadas do *data frame*.

In [ ]:
import pandas as pd

cut_d = pd.get_dummies(dds['cut'], dtype=float)
cut_d.head()

In [ ]:
color_d = pd.get_dummies(dds['color'], dtype=float)
color_d.head()

In [ ]:
clarity_d = pd.get_dummies(dds['clarity'], dtype=float)
clarity_d.head()

São criadas variáveis *dummy* para cada uma das possíveis respostas das variáveis categóricas. Para evitar multicolinearidade é interessante remover uma das resposta (que é função das demais).

In [ ]:
cut_d.drop(columns= 'Fair', inplace= True)
color_d.drop(columns= 'J', inplace= True)
clarity_d.drop(columns= 'I1', inplace= True)

### Adicionando as colunas *dummy* na matriz X

In [ ]:
fullX = pd.concat([X,
                   cut_d['Ideal'], cut_d['Premium'],
                   cut_d['Very Good'], cut_d['Good'],
                   color_d['D'], color_d['E'], color_d['F'],
                   color_d['G'], color_d['H'], color_d['I'],
                   clarity_d['IF'], clarity_d['VVS1'],
                   clarity_d['VVS2'], clarity_d['VS1'],
                   clarity_d['VS2'], clarity_d['SI1'], clarity_d['SI2']],
                  axis= 1)
fullX.head()

### Executando a regressão completa

In [ ]:
full_model = sm.OLS(Y, fullX)
full_model_res = full_model.fit()
full_model_res.summary()

### Usando fórmulas no estilo do *R*

Ao invés de criar o modelo com as variáveis *dummies* é possível executar o modelo com uma sintaxe similar ao usado no *R*:

In [ ]:
import statsmodels.formula.api as smf
model =  smf.ols(
    'logprice ~ carat + cut + color + clarity + x + y + z + depth + table + 1',
    data= dds)
model_res = model.fit()
model_res.summary()

### Verificando a colinearidade

#### Matriz de diagramas de pontos

A biblioteca *Seaborn* disponibiliza o método `pairplot()` para plotar matrizes de diagrams de pontos.

In [ ]:
import seaborn as sns
X = dds[['carat', 'x', 'y', 'z', 'depth', 'table']]
sns.pairplot(X);

#### Restringindo as variáveis na matriz de diagramas de pontos

Em um conjunto de dados com muitas colunas, a matriz de diagramas de pontos pode se tornar ilegível. Portanto, é possível escolher apenas algumas das colunas para exibir no gráfico.


In [ ]:
sns.pairplot(X[['x', 'y', 'z', 'table']]);

### Matriz de correlação

As matrizes de correlação podem ser úteis em diversos modelos (multivariados). Tal matriz podem ser calculadas de forma muito simples.

In [ ]:
round(dds.corr(numeric_only= True), 2)

In [ ]:
sns.heatmap(dds.corr(numeric_only= True), cmap='crest');

### Diagnóstico de regressão

É possível desenhar gráficos de qualidade de ajuste.

In [ ]:
from scipy import stats
sns.displot(model.fit().resid, kde= True);

In [ ]:
sns.boxplot(list(model.fit().resid), showmeans=True);

In [ ]:
sm.qqplot(model.fit().resid, line='s');

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

yy = model.fit().fittedvalues
temp = pd.concat([Y, yy.rename('fitted')], axis =1)

Y_max = Y.max()
Y_min = Y.min()

ax = sns.scatterplot(x = temp['fitted'], y = temp['logprice'])
ax.set(ylim=(Y_min, Y_max))
ax.set(xlim=(Y_min, Y_max))
ax.set_xlabel("Predicted value log price")
ax.set_ylabel("Observed value log price")

X_ref = Y_ref = np.linspace(Y_min, Y_max, 100)
plt.plot(X_ref, Y_ref, color='red', linewidth=1)
plt.show()